In [1]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
def draw_landmarks_on_image(rgb_image, detection_result):
    face_landmarks_list = detection_result.face_landmarks
    annotated_image = np.copy(rgb_image)
    for idx in range(len(face_landmarks_list)):
        face_landmarks = face_landmarks_list[idx]
        face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        face_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
        ])
        # ציור הנקודות והחיבורים
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_tesselation_style())
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_contours_style())
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_iris_connections_style())
        # ✅ 473,263,362הוספת מספרים למספר נקודות בו-זמנית
        landmark_indices = [] # רשימת האינדקסים שברצונך להציג
        for landmark_index in landmark_indices:
            if landmark_index < len(face_landmarks):
                landmark = face_landmarks[landmark_index]
                h, w, _ = annotated_image.shape  # גודל התמונה
                cx, cy = int(landmark.x * w), int(landmark.y * h)  # המרת קואורדינטות יחסיות לפיקסלים
                # הגדרות פונט קטן ומדויק
                text = str(landmark_index)
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.25  # מספר קטן מאוד
                thickness = 1
                text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
                # חישוב נקודת ההתחלה - מתחת לנקודה כדי לא להסתיר אותה
                text_x = cx - text_size[0] //2
                text_y = cy + text_size[1] +2# מזיז מעט מתחת לנקודה
                # ✅ הוספת רקע שחור מאחורי המספר לשיפור הבהירות
                cv2.putText(annotated_image, text, (text_x, text_y), font, font_scale, (0, 0, 0), thickness + 2, cv2.LINE_AA)  # רקע שחור
                cv2.putText(annotated_image, text, (text_x, text_y), font, font_scale, (0, 255, 0), thickness, cv2.LINE_AA)  # טקסט ירוק
    return annotated_image
# פתיחת מצלמה
cap = cv2.VideoCapture(0)  # 0 עבור מצלמה ראשית
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()
# קריאת פריימים מהמצלמה
print("Press 's' to save the image and exit, or 'q' to exit without saving.")
while True:
    ret, frame = cap.read()  # קריאת פריים מהמצלמה
    if not ret:
        print("Failed to grab frame.")
        break
    cv2.imshow("Camera", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):  # שמירת תמונה
        cv2.imwrite("captured_image.png", frame)
        print("Image saved as 'ca,ptured_image.png'")
        break
    elif key == ord('q'):  # יציאה
        print("Exiting without saving.")
        break
cap.release()
cv2.destroyAllWindows()
# קריאת התמונה שנשמרה
image_format = mp.ImageFormat.SRGB
image = mp.Image(image_format, np.array(frame))
# יצירת מודל לזיהוי נקודות בפנים
base_options = python.BaseOptions(model_asset_path=r'C:\face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)
# זיהוי נקודות הפנים בתמונה
detection_result = detector.detect(image)
# ציור הנקודות והוספת המספרים
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
# הצגת התמונה עם המספרים
cv2.imshow("Annotated Image", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()
print(((1-detection_result.face_landmarks[0][263].y)-(1-detection_result.face_landmarks[0][359].y))/((detection_result.face_landmarks[0][263].x-detection_result.face_landmarks[0][359].x)))
print(detection_result.face_landmarks[0][263].y)
print(detection_result.face_landmarks[0][359].y)
print(detection_result.face_landmarks[0][263].x)
print(detection_result.face_landmarks[0][359].x)


Press 's' to save the image and exit, or 'q' to exit without saving.
Image saved as 'ca,ptured_image.png'
0.12518852218196064
0.4254525303840637
0.4246906042098999
0.7979944944381714
0.8040807247161865


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import time
import os

# הגדרות FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False)

# פונקציה לזיהוי נקודות חשובות בפנים
def get_points(landmarks, w, h):
    return np.array([
        [landmarks[1].x * w, landmarks[1].y * h],     # אף
        [landmarks[33].x * w, landmarks[33].y * h],   # עין שמאל
        [landmarks[263].x * w, landmarks[263].y * h], # עין ימין
        [landmarks[61].x * w, landmarks[61].y * h],   # פה שמאל
        [landmarks[291].x * w, landmarks[291].y * h], # פה ימין
        [landmarks[199].x * w, landmarks[199].y * h], # סנטר
    ], dtype=np.float64)

model_points = np.array([
    [0.0, 0.0, 0.0],
    [-30.0, -30.0, -30.0],
    [30.0, -30.0, -30.0],
    [-25.0, 30.0, -30.0],
    [25.0, 30.0, -30.0],
    [0.0, 70.0, -50.0],
])

# חישוב זוויות הראש
def estimate_angles(frame):
    h, w, _ = frame.shape
    results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
        return None
    landmarks = results.multi_face_landmarks[0].landmark
    image_points = get_points(landmarks, w, h)

    focal_length = w
    center = (w / 2, h / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")

    dist_coeffs = np.zeros((4, 1))
    _, rvec, tvec = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)
    rotation_mat, _ = cv2.Rodrigues(rvec)
    angles, _, _, _, _, _ = cv2.RQDecomp3x3(rotation_mat)
    return angles  # pitch, yaw, roll

# בדיקת ישרות
def is_straight(pitch, yaw, roll, tol=12):
    return abs(pitch) < tol and abs(yaw) < tol and abs(roll) < tol

# ציור מסגרת עזר
# def draw_center_box(frame):
#     h, w = frame.shape[:2]
#     box_w, box_h = int(w * 0.4), int(h * 0.6)
#     x1, y1 = (w - box_w) // 2, (h - box_h) // 2
#     x2, y2 = x1 + box_w, y1 + box_h
#     cv2.rectangle(frame, (x1, y1), (x2, y2), (200, 255, 200), 2)
#     return (x1, y1, x2, y2)

# תיעוד אוטומטי
def auto_capture(frame):
    ts = int(time.time())
    filename = f"face_capture_{ts}.jpg"
    cv2.imwrite(filename, frame)
    print(f"✔ נשמרה תמונה: {filename}")

# התחלת צילום וידאו
cap = cv2.VideoCapture(0)
captured = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    angles = estimate_angles(frame)
    draw_center_box(frame)

    if angles:
        pitch, yaw, roll = angles
        aligned = is_straight(pitch, yaw, roll)

        status = f"P={pitch:.1f} Y={yaw:.1f} R={roll:.1f}"
        color = (0, 255, 0) if aligned else (0, 0, 255)
        msg = "✅ ישר - מצלם!" if aligned else "❌ לא ישר"

        cv2.putText(frame, status + " " + msg, (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        if aligned and not captured:
            auto_capture(frame)
            captured = True
            break
    cv2.imshow("מצלמה - יישור פנים", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('r'):
        captured = False  # איפוס צילום

cap.release()
cv2.destroyAllWindows()


NameError: name 'draw_center_box' is not defined

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
from tkinter import filedialog
import time

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

model_points = np.array([
    [0.0, 0.0, 0.0],
    [-30.0, -30.0, -30.0],
    [30.0, -30.0, -30.0],
    [-25.0, 30.0, -30.0],
    [25.0, 30.0, -30.0],
    [0.0, 70.0, -50.0],
])

def get_image_points(landmarks, w, h):
    return np.array([
        [landmarks[1].x * w, landmarks[1].y * h],
        [landmarks[33].x * w, landmarks[33].y * h],
        [landmarks[263].x * w, landmarks[263].y * h],
        [landmarks[61].x * w, landmarks[61].y * h],
        [landmarks[291].x * w, landmarks[291].y * h],
        [landmarks[199].x * w, landmarks[199].y * h],
    ], dtype=np.float64)

def estimate_angles(image):
    h, w = image.shape[:2]
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
        return None
    landmarks = results.multi_face_landmarks[0].landmark
    image_points = get_image_points(landmarks, w, h)

    focal_length = w
    center = (w / 2, h / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")

    dist_coeffs = np.zeros((4, 1))
    _, rvec, _ = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)
    rotation_mat, _ = cv2.Rodrigues(rvec)
    angles, _, _, _, _, _ = cv2.RQDecomp3x3(rotation_mat)
    return angles  # pitch, yaw, roll

def is_straight(pitch, yaw, roll, tol=15):  # הגדלת טולרנס
    # בטולרנס גבוה יותר נוכל להתאים את זה למציאות
    return abs(pitch) < tol and abs(yaw) < tol and abs(roll) < tol

def draw_center_box(frame):
    h, w = frame.shape[:2]
    box_w, box_h = int(w * 0.4), int(h * 0.6)
    x1, y1 = (w - box_w) // 2, (h - box_h) // 2
    x2, y2 = x1 + box_w, y1 + box_h
    cv2.rectangle(frame, (x1, y1), (x2, y2), (100, 255, 100), 2)

def process_image(image):
    angles = estimate_angles(image)
    if angles:
        pitch, yaw, roll = angles
        straight = is_straight(pitch, yaw, roll)
        print(f"Pitch={pitch:.1f}, Yaw={yaw:.1f}, Roll={roll:.1f}, Straight={straight}")
        if straight:
            filename = f"valid_face_{int(time.time())}.jpg"
            cv2.imwrite(filename, image)
            print(f"✔ תמונה נשמרה: {filename}")
        else:
            print("❌ התמונה לא ישרה")
    else:
        print("❌ לא זוהה פנים")

def upload_image():
    path = filedialog.askopenfilename()
    if path:
        image = cv2.imread(path)
        process_image(image)

def capture_from_camera():
    face_mesh_camera = mp_face_mesh.FaceMesh(static_image_mode=False)
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        draw_center_box(frame)
        angles = estimate_angles(frame)
        if angles:
            pitch, yaw, roll = angles
            aligned = is_straight(pitch, yaw, roll)
            color = (0, 255, 0) if aligned else (0, 0, 255)
            msg = "✅ מוכן לצילום" if aligned else "❌ לא מוכן"
            cv2.putText(frame, msg, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            if aligned:
                filename = f"captured_face_{int(time.time())}.jpg"
                cv2.imwrite(filename, frame)
                print(f"✔ תמונה נשמרה: {filename}")
                break

        cv2.imshow("מצלמה", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# GUI לבחירה
root = tk.Tk()
root.title("בחירת מקור תמונה")
root.geometry("300x100")

btn1 = tk.Button(root, text="📷 צילום במצלמה", command=capture_from_camera)
btn1.pack(pady=10)

btn2 = tk.Button(root, text="🖼 העלאת תמונה מהמחשב", command=upload_image)
btn2.pack()

root.mainloop()


✔ תמונה נשמרה: captured_face_1746577220.jpg
✔ תמונה נשמרה: captured_face_1746577234.jpg
✔ תמונה נשמרה: captured_face_1746577242.jpg
✔ תמונה נשמרה: captured_face_1746577248.jpg
✔ תמונה נשמרה: captured_face_1746577257.jpg
Pitch=-16.0, Yaw=-0.3, Roll=-0.0, Straight=False
❌ התמונה לא ישרה
Pitch=-40.3, Yaw=-1.4, Roll=1.6, Straight=False
❌ התמונה לא ישרה
Pitch=-5.4, Yaw=-2.0, Roll=0.3, Straight=True
✔ תמונה נשמרה: valid_face_1746577285.jpg
Pitch=-13.0, Yaw=-0.3, Roll=-0.3, Straight=True
✔ תמונה נשמרה: valid_face_1746577294.jpg
Pitch=-32.4, Yaw=8.2, Roll=-1.1, Straight=False
❌ התמונה לא ישרה
Pitch=-30.1, Yaw=1.6, Roll=2.1, Straight=False
❌ התמונה לא ישרה
Pitch=-27.9, Yaw=-1.3, Roll=-2.6, Straight=False
❌ התמונה לא ישרה
Pitch=43.3, Yaw=-2.3, Roll=-2.0, Straight=False
❌ התמונה לא ישרה
Pitch=39.1, Yaw=0.1, Roll=-4.2, Straight=False
❌ התמונה לא ישרה
Pitch=-23.2, Yaw=0.3, Roll=-2.1, Straight=False
❌ התמונה לא ישרה
Pitch=-24.5, Yaw=0.3, Roll=0.1, Straight=False
❌ התמונה לא ישרה
Pitch=23.6, Yaw=-0.4

✔ תמונה נשמרה: captured_face_1746577752.jpg
✔ תמונה נשמרה: captured_face_1746577774.jpg
✔ תמונה נשמרה: captured_face_1746577781.jpg
✔ תמונה נשמרה: captured_face_1746577786.jpg
✔ תמונה נשמרה: captured_face_1746577789.jpg
Pitch=-22.4, Yaw=-2.2, Roll=-2.7, LookingForward=False
❌ הפנים לא ישרות או המבט לא קדימה
Pitch=-24.1, Yaw=-0.7, Roll=-0.6, LookingForward=False
❌ הפנים לא ישרות או המבט לא קדימה
Pitch=-5.6, Yaw=-1.9, Roll=0.2, LookingForward=True
✔ תמונה נשמרה: valid_face_1746577814.jpg
Pitch=-12.7, Yaw=-0.4, Roll=-0.4, LookingForward=False
❌ הפנים לא ישרות או המבט לא קדימה
Pitch=15.2, Yaw=-1.4, Roll=-0.9, LookingForward=False
❌ הפנים לא ישרות או המבט לא קדימה
Pitch=-38.5, Yaw=-0.9, Roll=-3.3, LookingForward=False
❌ הפנים לא ישרות או המבט לא קדימה
Pitch=23.0, Yaw=0.5, Roll=0.3, LookingForward=False
❌ הפנים לא ישרות או המבט לא קדימה
Pitch=0.2, Yaw=-1.5, Roll=-0.2, LookingForward=True
✔ תמונה נשמרה: valid_face_1746577851.jpg
Pitch=-23.6, Yaw=1.4, Roll=0.3, LookingForward=False
❌ הפנים לא 

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
from tkinter import filedialog
import time

mp_face_mesh = mp.solutions.face_mesh
face_mesh_static = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)
face_mesh_camera = mp_face_mesh.FaceMesh(static_image_mode=False, refine_landmarks=True)

# נקודות תלת ממדיות של הראש
model_points = np.array([
    [0.0, 0.0, 0.0],
    [-30.0, -30.0, -30.0],
    [30.0, -30.0, -30.0],
    [-25.0, 30.0, -30.0],
    [25.0, 30.0, -30.0],
    [0.0, 70.0, -50.0],
])

def get_image_points(landmarks, w, h):
    return np.array([
        [landmarks[1].x * w, landmarks[1].y * h],
        [landmarks[33].x * w, landmarks[33].y * h],
        [landmarks[263].x * w, landmarks[263].y * h],
        [landmarks[61].x * w, landmarks[61].y * h],
        [landmarks[291].x * w, landmarks[291].y * h],
        [landmarks[199].x * w, landmarks[199].y * h],
    ], dtype=np.float64)

def estimate_angles(image, mesh_model):
    h, w = image.shape[:2]
    results = mesh_model.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
        return None, None
    landmarks = results.multi_face_landmarks[0].landmark
    image_points = get_image_points(landmarks, w, h)
    focal_length = w
    center = (w / 2, h / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")
    dist_coeffs = np.zeros((4, 1))
    _, rvec, _ = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)
    rotation_mat, _ = cv2.Rodrigues(rvec)
    angles, _, _, _, _, _ = cv2.RQDecomp3x3(rotation_mat)
    return angles, landmarks

def is_straight(pitch, yaw, roll, tol=10):
    return abs(pitch) < tol and abs(yaw) < tol and abs(roll) < tol

def is_gaze_forward(landmarks, w):
    try:
        # עין ימין
        re_left = landmarks[33].x * w
        re_right = landmarks[133].x * w
        re_center = landmarks[468].x * w
        re_ratio = (re_center - re_left) / (re_right - re_left)

        # עין שמאל
        le_left = landmarks[362].x * w
        le_right = landmarks[263].x * w
        le_center = landmarks[473].x * w
        le_ratio = (le_center - le_left) / (le_right - le_left)

        return 0.35 < re_ratio < 0.65 and 0.35 < le_ratio < 0.65
    except:
        return False

def draw_center_box(frame):
    h, w = frame.shape[:2]
    box_w, box_h = int(w * 0.4), int(h * 0.6)
    x1, y1 = (w - box_w) // 2, (h - box_h) // 2
    x2, y2 = x1 + box_w, y1 + box_h
    cv2.rectangle(frame, (x1, y1), (x2, y2), (100, 255, 100), 2)

def process_image(image):
    h, w = image.shape[:2]
    angles, landmarks = estimate_angles(image, face_mesh_static)
    if angles and landmarks:
        pitch, yaw, roll = angles
        straight = is_straight(pitch, yaw, roll)
        gaze = is_gaze_forward(landmarks, w)
        print(f"Pitch={pitch:.1f}, Yaw={yaw:.1f}, Roll={roll:.1f}, Straight={straight}, Gaze={gaze}")
        if straight and gaze:
            filename = f"valid_face_{int(time.time())}.jpg"
            cv2.imwrite(filename, image)
            print(f"✔ תמונה נשמרה: {filename}")
        else:
            print("❌ התמונה לא ישרה או שהמבט לא למצלמה")
    else:
        print("❌ לא זוהה פנים")

def upload_image():
    path = filedialog.askopenfilename()
    if path:
        image = cv2.imread(path)
        process_image(image)

def capture_from_camera():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        draw_center_box(frame)
        h, w = frame.shape[:2]
        angles, landmarks = estimate_angles(frame, face_mesh_camera)
        if angles and landmarks:
            pitch, yaw, roll = angles
            straight = is_straight(pitch, yaw, roll)
            gaze = is_gaze_forward(landmarks, w)
            color = (0, 255, 0) if straight and gaze else (0, 0, 255)
            msg = "✅ מוכן לצילום" if straight and gaze else "❌ לא מוכן"
            cv2.putText(frame, msg, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            if straight and gaze:
                filename = f"captured_face_{int(time.time())}.jpg"
                cv2.imwrite(filename, frame)
                print(f"✔ תמונה נשמרה: {filename}")
                break
        cv2.imshow("מצלמה", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# GUI
root = tk.Tk()
root.title("בחירת מקור תמונה")
root.geometry("300x100")

btn1 = tk.Button(root, text="📷 צילום במצלמה", command=capture_from_camera)
btn1.pack(pady=10)

btn2 = tk.Button(root, text="🖼 העלאת תמונה מהמחשב", command=upload_image)
btn2.pack()

root.mainloop()


✔ תמונה נשמרה: captured_face_1746609033.jpg
✔ תמונה נשמרה: captured_face_1746609040.jpg
✔ תמונה נשמרה: captured_face_1746609048.jpg
✔ תמונה נשמרה: captured_face_1746609093.jpg
✔ תמונה נשמרה: captured_face_1746609101.jpg
✔ תמונה נשמרה: captured_face_1746609110.jpg
✔ תמונה נשמרה: captured_face_1746609124.jpg
✔ תמונה נשמרה: captured_face_1746609131.jpg
✔ תמונה נשמרה: captured_face_1746609142.jpg
✔ תמונה נשמרה: captured_face_1746609152.jpg
✔ תמונה נשמרה: captured_face_1746609157.jpg
✔ תמונה נשמרה: captured_face_1746609167.jpg
Pitch=-7.8, Yaw=6.9, Roll=-3.7, Straight=True, Gaze=True
✔ תמונה נשמרה: valid_face_1746609182.jpg
Pitch=1.7, Yaw=-1.0, Roll=0.3, Straight=True, Gaze=True
✔ תמונה נשמרה: valid_face_1746609190.jpg
Pitch=-7.5, Yaw=6.8, Roll=-3.6, Straight=True, Gaze=True
✔ תמונה נשמרה: valid_face_1746609202.jpg
Pitch=-24.9, Yaw=2.2, Roll=179.2, Straight=False, Gaze=False
❌ התמונה לא ישרה או שהמבט לא למצלמה
Pitch=-27.0, Yaw=9.4, Roll=-0.6, Straight=False, Gaze=True
❌ התמונה לא ישרה או שהמ